In [1]:
import numpy as np
import pandas as pd
import requests
import json
import lxml
import unidecode
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from datetime import datetime

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

## 🇫🇮 Finland Cities Database

In [2]:
cities_df = pd.read_csv('https://simplemaps.com/static/data/country-cities/fi/fi.csv')

In [3]:
#remove accents
cols = cities_df.select_dtypes(include=[np.object]).columns
cities_df[cols] = cities_df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
cities_df

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Helsinki,60.1756,24.9342,Finland,FI,Uusimaa,primary,642045.0,642045.0
1,Espoo,60.2100,24.6600,Finland,FI,Uusimaa,minor,269802.0,269802.0
2,Tampere,61.4981,23.7608,Finland,FI,Pirkanmaa,admin,225118.0,225118.0
3,Vantaa,60.3000,25.0333,Finland,FI,Uusimaa,minor,214605.0,214605.0
4,Oulu,65.0142,25.4719,Finland,FI,Pohjois-Pohjanmaa,admin,200526.0,200526.0
...,...,...,...,...,...,...,...,...,...
321,Karlby,59.9167,20.9000,Finland,FI,Aland,minor,NaN,NaN
322,Polvijarvi,62.4167,28.3333,Finland,FI,Etela-Savo,minor,NaN,NaN
323,Kolari,67.3317,23.7913,Finland,FI,Lappi,minor,NaN,NaN
324,Ivalo,68.6565,27.5404,Finland,FI,Lappi,minor,NaN,NaN


In [4]:
# shape and data types of the data
print(cities_df.shape)
print(cities_df.dtypes)

(326, 9)
city                  object
lat                  float64
lng                  float64
country               object
iso2                  object
admin_name            object
capital               object
population           float64
population_proper    float64
dtype: object


In [209]:
#Display the percentage of missing data in the dataset

for col in cities_df.columns:
    pct_missing = np.mean(cities_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))


city - 0.0%
lat - 0.0%
lng - 0.0%
country - 0.0%
iso2 - 0.0%
admin_name - 0.0%
capital - 1.0%
population - 8.0%
population_proper - 8.0%


## 🇫🇮 Port-calls Database 

In [6]:
## Data contains Portnet Port Call information such as time of arrival and departure, 
## ship name and name of the agent. API can deliver real-time information and future schedules. 

# no more than 1000 items per call
# we have data available only from 9.10.2018 onwards

datelist = pd.date_range(start = '2022-07-01', end = datetime.today()).astype(str).tolist()

port_calls = []

for timestamp in datelist:
    url = f'https://meri.digitraffic.fi/api/v1/port-calls?date={timestamp}'
    api_response = requests.get(url).text
    response_info = json.loads(api_response)
    for call_info in response_info['portCalls']:
        port_calls.append([call_info['portCallId'], call_info['portCallTimestamp'], call_info['customsReference'],
                          call_info['portToVisit'], call_info['prevPort'], call_info['nextPort'],
                          call_info['domesticTrafficArrival'], call_info['domesticTrafficDeparture'], call_info['arrivalWithCargo'],
                          call_info['notLoading'], call_info['discharge'],
                          call_info['vesselName'], call_info['vesselNamePrefix'], call_info['radioCallSign'],
                          call_info['radioCallSignType'], call_info['imoLloyds'], call_info['mmsi'],
                          call_info['nationality'], call_info['vesselTypeCode'], call_info['certificateIssuer'],
                          call_info['certificateStartDate'], call_info['certificateEndDate'], call_info['currentSecurityLevel'],
                          call_info['agentInfo'], response_info['portCallsUpdated'], timestamp])

    
port_calls_df = pd.DataFrame(data=port_calls, columns=['portCallId', 'portCallTimestamp', 'customsReference',
                                                      'portToVisit', 'prevPort', 'nextPort',
                                                      'domesticTrafficArrival', 'domesticTrafficDeparture', 'arrivalWithCargo',
                                                      'notLoading', 'discharge',
                                                      'vesselName', 'vesselNamePrefix', 'radioCallSign',
                                                      'radioCallSignType', 'imoLloyds', 'mmsi',
                                                      'nationality', 'vesselTypeCode', 'certificateIssuer',
                                                      'certificateStartDate', 'certificateEndDate', 'currentSecurityLevel',
                                                      'agentInfo', 'portCallsUpdated', 'calltimestamp'])

port_calls_df

,portCallId,portCallTimestamp,customsReference,portToVisit,prevPort,nextPort,domesticTrafficArrival,domesticTrafficDeparture,arrivalWithCargo,notLoading,...,mmsi,nationality,vesselTypeCode,certificateIssuer,certificateStartDate,certificateEndDate,currentSecurityLevel,agentInfo,portCallsUpdated,calltimestamp
0,2812847,2022-07-01T07:15:35.000+00:00,16/1604928,FIPOR,SEOXE,FIHEL,False,False,True,False,...,230683000,FI,60,FI/Traficom,2019-04-05T21:00:00.000+00:00,2024-06-04T21:00:00.000+00:00,1,"[{'name': 'Oy M Rauanheimo Ab', 'role': 1, 'po...",2022-08-03T20:54:00.184728Z,2022-07-01
1,2813751,2022-07-01T11:30:34.000+00:00,16/1604929,FIRAA,TRIZM,FIRAA,False,False,True,True,...,305718000,AG,70,Antigua Barbuda /DNV,2022-01-28T22:00:00.000+00:00,2023-10-01T21:00:00.000+00:00,1,[{'name': 'BBC Chartering & Logistic Gmbh && C...,2022-08-03T20:54:00.184728Z,2022-07-01
2,2825443,2022-07-01T19:30:59.000+00:00,16/1604975,FIKOK,ESHUV,FIPRS,False,False,True,True,...,305716000,PT,70,PT/RINA,2021-06-05T21:00:00.000+00:00,2024-10-14T21:00:00.000+00:00,1,"[{'name': 'Dalaro Shipping Ltd', 'role': 2, 'p...",2022-08-03T20:54:00.184728Z,2022-07-01
3,2836891,2022-07-01T18:03:29.000+00:00,0/669673,FITKU,FIMHQ,FILAN,False,False,True,False,...,230629000,FI,20,Finnish Transport Safety Agency,2018-06-10T21:00:00.000+00:00,2023-06-09T21:00:00.000+00:00,1,"[{'name': 'Viking Line Abp', 'role': 2, 'port...",2022-08-03T20:54:00.184728Z,2022-07-01
4,2840542,2022-07-01T05:56:16.000+00:00,0/670087,FIECK,SEGRH,SEGRH,False,False,True,False,...,266308000,SE,20,SE/BV,2019-03-12T22:00:00.000+00:00,2024-03-15T22:00:00.000+00:00,1,"[{'name': 'Eckerö Linjen Ab', 'role': 1, 'port...",2022-08-03T20:54:00.184728Z,2022-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,2881533,2022-08-03T15:01:18.000+00:00,0/676100,FINLI,FISKV,FINLI,True,True,True,True,...,230957000,FI,80,FI/Finnish Transport,2020-08-31T21:00:00.000+00:00,2025-04-09T21:00:00.000+00:00,1,"[{'name': 'C & C Port Agency Finland Oy Ltd, H...",2022-08-03T20:54:00.184728Z,2022-08-03
4739,2881539,2022-08-03T18:38:05.000+00:00,16/1605196,FIPOR,DERSK,RUULU,False,False,True,False,...,249624000,AG,70,AG/Rina,2022-04-04T21:00:00.000+00:00,2022-09-03T21:00:00.000+00:00,1,"[{'name': 'Oy M Rauanheimo Ab', 'role': 1, 'po...",2022-08-03T20:54:00.184728Z,2022-08-03
4740,2881542,2022-08-03T16:15:04.000+00:00,0/676101,FIANK,SETUN,USBAL,False,False,True,True,...,244750715,NL,60,NL/LR,2018-11-19T22:00:00.000+00:00,2023-12-07T22:00:00.000+00:00,1,"[{'name': 'Euroports Rauma Oy', 'role': 1, 'po...",2022-08-03T20:54:00.184728Z,2022-08-03
4741,2881549,2022-08-03T18:41:34.000+00:00,14/80026376,FIKTK,FISKV,LVRIX,False,False,True,False,...,247102500,IT,80,IT/Rina,2019-05-16T21:00:00.000+00:00,2024-06-14T21:00:00.000+00:00,1,"[{'name': 'GAC Finland Oy', 'role': 1, 'portCa...",2022-08-03T20:54:00.184728Z,2022-08-03


In [220]:
# Look at # of records, and data types
print(port_calls_df.shape)
print(port_calls_df.dtypes)

(4037, 26)
portCallId                    int64
portCallTimestamp            object
customsReference             object
portToVisit                  object
prevPort                     object
nextPort                     object
domesticTrafficArrival         bool
domesticTrafficDeparture       bool
arrivalWithCargo               bool
notLoading                     bool
discharge                     int64
vesselName                   object
vesselNamePrefix             object
radioCallSign                object
radioCallSignType            object
imoLloyds                   float64
mmsi                          int64
nationality                  object
vesselTypeCode                int64
certificateIssuer            object
certificateStartDate         object
certificateEndDate           object
currentSecurityLevel          int64
agentInfo                    object
portCallsUpdated             object
calltimestamp                object
dtype: object


In [221]:
# Look at missing values

for col in port_calls_df.columns:
    pct_missing = np.mean(port_calls_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

portCallId - 0.0%
portCallTimestamp - 0.0%
customsReference - 0.0%
portToVisit - 0.0%
prevPort - 0.0%
nextPort - 0.0%
domesticTrafficArrival - 0.0%
domesticTrafficDeparture - 0.0%
arrivalWithCargo - 0.0%
notLoading - 0.0%
discharge - 0.0%
vesselName - 0.0%
vesselNamePrefix - 0.0%
radioCallSign - 0.0%
radioCallSignType - 0.0%
imoLloyds - 0.0%
mmsi - 0.0%
nationality - 0.0%
vesselTypeCode - 0.0%
certificateIssuer - 0.0%
certificateStartDate - 0.0%
certificateEndDate - 0.0%
currentSecurityLevel - 0.0%
agentInfo - 0.0%
portCallsUpdated - 0.0%
calltimestamp - 0.0%


In [217]:
# portCallID is a unique column
port_calls_df['portCallId'].duplicated().any()

False

## 🇫🇮 Code for Trade and Transport Locations

In [7]:
locode = pd.read_html('http://service.unece.org/trade/locode/fi.htm', match = 'NameWoDiacritics', header = 0)
locode_df = locode[0]

In [8]:
#check data types of the dataset
print(locode_df.shape)
print(locode_df.dtypes)

(784, 11)
Ch                   object
LOCODE               object
Name                 object
NameWoDiacritics     object
SubDiv              float64
Function             object
Status               object
Date                float64
IATA                 object
Coordinates          object
Remarks             float64
dtype: object


In [224]:
#display missing values in percentages
for col in locode_df.columns:
    pct_missing = np.mean(locode_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Ch - 99.0%
LOCODE - 0.0%
Name - 0.0%
NameWoDiacritics - 0.0%
SubDiv - 6.0%
Function - 1.0%
Status - 1.0%
Date - 1.0%
IATA - 98.0%
Coordinates - 7.0%
Remarks - 100.0%


In [225]:
#splitting coordinates column into two columns
lat = []
lon = []


for row in locode_df['Coordinates']:
    try:
        lat.append(row.split(' ')[0])
        lon.append(row.split(' ')[1])
    except:
        lat.append(np.NaN)
        lon.append(np.NaN)


locode_df['latitude'] = lat
locode_df['longitude'] = lon

In [227]:
#remove space in LOCODE to unify the standard
locode_df['LOCODE'] = locode_df['LOCODE'].str.replace(" ","")

locode_df

,Ch,LOCODE,Name,NameWoDiacritics,SubDiv,Function,Status,Date,IATA,Coordinates,Remarks,latitude,longitude
0,NaN,FIAAN,Äänekoski,Aanekoski,8.0,--3-----,AF,1307.0,NaN,NaN,NaN,NaN,NaN
1,NaN,FIAVS,Aavasaksa,Aavasaksa,10.0,--3----B,AC,1307.0,NaN,NaN,NaN,NaN,NaN
2,NaN,FITKU,Åbo (Turku),Abo (Turku),19.0,1-34----,AI,1307.0,NaN,6027N 02217E,NaN,6027N,02217E
3,NaN,FIATS,Äetsä,Aetsa,11.0,-23-----,RL,1307.0,NaN,6117N 02241E,NaN,6117N,02241E
4,NaN,FIATI,Ähtäri,Ahtari,NaN,--3-----,RQ,101.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,NaN,FIYTO,Ylitornio,Ylitornio,15.0,--3-----,AA,2007.0,NaN,6619N 02340E,NaN,6619N,02340E
780,NaN,FIYLI,Ylivieska,Ylivieska,14.0,---4----,AF,1401.0,NaN,6405N 02433E,NaN,6405N,02433E
781,NaN,FIYJA,Ylöjärvi,Ylojarvi,11.0,-23--6--,RL,1307.0,NaN,6133N 02335E,NaN,6133N,02335E
782,NaN,FIYPA,Ypäjä,Ypaja,19.0,--3-----,AA,1907.0,NaN,6048N 02317E,NaN,6048N,02317E


## All Finnish registered ships

In [9]:
ships_df = pd.read_csv('/Users/keepitdan/Desktop/Untitled Folder/Alusrekisteri1.csv', sep=';')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/keepitdan/Desktop/Untitled Folder/Alusrekisteri1.csv'

In [231]:
#check data types of the dataset
print(ships_df.shape)
print(ships_df.dtypes)

(2574, 23)
alus_id                   int64
nimi                     object
alustyyppi                int64
Tilakoodi                object
Ensirekisterointipvm     object
Uusikotimaa             float64
Vanhakotimaa            float64
Rakennusvuosi           float64
Rakennusmaa             float64
Rakennuspaikka           object
omistajuustyyppi          int64
asiakkaan_kuntatieto    float64
aluksen_satamapaikka     object
Kantavuus               float64
JaaluokkaKoodi           object
Nettovetoisuus          float64
Bruttovetoisuus         float64
Pituus                   object
Suurinpituus             object
Mallisyvays              object
Mallikorkeus             object
Leveys                   object
Unnamed: 22             float64
dtype: object


In [236]:
ships_df.rename(columns = {'alus_id':'vessel_id', 'nimi':'name', 'alustyyppi':'vessel_type',
                           'Tilakoodi':'status_code', 'Ensirekisterointipvm':'first_registration_date', 
                           'Uusikotimaa':'new_homeland', 'Vanhakotimaa':'old_homeland', 
                           'Rakennusvuosi':'constrution_year', 'Rakennusmaa':'constrution_country',
                           'Rakennuspaikka':'construction_type', 'omistajuustyyppi':'ownership_type',
                           'asiakkaan_kuntatieto':'customer_municipality', 'aluksen_satamapaikka':'vessel_port',
                           'Kantavuus':'cattying_capacity', 'JaaluokkaKoodi':'subcategory_code',
                           'Nettovetoisuus':'net_weight', 'Bruttovetoisuus':'gross_tonnage',
                           'Pituus':'length', 'Suurinpituus':'maximum_length', 'Mallisyvays':'model_depth',
                           'Mallikorkeus':'model_height', 'Leveys':'width'
                          }, inplace = True)

ships_df

,vessel_id,name,vessel_type,status_code,first_registration_date,new_homeland,old_homeland,constrution_year,constrution_country,construction_type,...,cattying_capacity,subcategory_code,net_weight,gross_tonnage,length,maximum_length,model_depth,model_height,width,Unnamed: 22
0,1,Arctica,4,VESSEL_REGISTERED_FI,17.01.1989,NaN,246.0,NaN,NaN,NaN,...,NaN,NaN,57.0,187.0,"30,00",NaN,"2,63","3,50","7,16",NaN
1,1,Arctica,4,VESSEL_REGISTERED_FI,17.01.1989,NaN,246.0,1967.0,100.0,Burgas,...,NaN,NaN,57.0,187.0,"30,00",NaN,"2,63","3,50","7,16",NaN
2,2,Orjaku,11,VESSEL_REGISTERED_FI,10.05.1990,NaN,246.0,NaN,NaN,NaN,...,NaN,NaN,33.0,109.0,"26,40",NaN,"2,28","3,04","6,50",NaN
3,2,Orjaku,11,VESSEL_REGISTERED_FI,10.05.1990,NaN,246.0,1955.0,278.0,Magdenburg,...,NaN,NaN,33.0,109.0,"26,40",NaN,"2,28","3,04","6,50",NaN
4,3,Lyra,11,VESSEL_REGISTERED_FI,25.06.1973,NaN,752.0,NaN,NaN,NaN,...,500.0,NaN,144.0,258.0,"48,50",NaN,"2,43","2,72","7,00",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,1118,FBP KARHU,9,VESSEL_REGISTERED_FI,08.03.2022,NaN,999.0,NaN,NaN,NaN,...,NaN,NaN,9.0,28.0,"11,58",NaN,"1,07","1,42","7,16",NaN
2570,1118,FBP KARHU,9,VESSEL_REGISTERED_FI,08.03.2022,NaN,999.0,2022.0,246.0,Pori,...,NaN,NaN,9.0,28.0,"11,58",NaN,"1,07","1,42","7,16",NaN
2571,1119,FBP MAININKI,9,VESSEL_REGISTERED_FI,29.03.2022,NaN,246.0,NaN,NaN,NaN,...,NaN,NaN,12.0,37.0,"14,42",NaN,"1,15","1,53","6,84",NaN
2572,1119,FBP MAININKI,9,VESSEL_REGISTERED_FI,29.03.2022,NaN,246.0,NaN,NaN,NaN,...,NaN,NaN,12.0,37.0,"14,42",NaN,"1,15","1,53","6,84",NaN


In [239]:
# dupliacte values due to ownershiptype, maybe a good idea to divide dataset
ships_df.loc[ships_df['name']=='JATULI']

,vessel_id,name,vessel_type,status_code,first_registration_date,new_homeland,old_homeland,constrution_year,constrution_country,construction_type,...,cattying_capacity,subcategory_code,net_weight,gross_tonnage,length,maximum_length,model_depth,model_height,width,Unnamed: 22
2510,1093,JATULI,12,VESSEL_REGISTERED_FI,02.12.2021,NaN,410.0,NaN,NaN,NaN,...,NaN,IA,33498.0,63532.0,"241,30",NaN,"15,10","21,40","44,00",NaN
2511,1093,JATULI,12,VESSEL_REGISTERED_FI,02.12.2021,NaN,410.0,NaN,NaN,NaN,...,NaN,IA,33498.0,63532.0,"241,30",NaN,"15,10","21,40","44,00",NaN
2512,1093,JATULI,12,VESSEL_REGISTERED_FI,02.12.2021,NaN,410.0,NaN,NaN,NaN,...,NaN,IA,33498.0,63532.0,"241,30",NaN,"15,10","21,40","44,00",NaN
2513,1093,JATULI,12,VESSEL_REGISTERED_FI,02.12.2021,NaN,410.0,2021.0,410.0,Ulsan,...,NaN,IA,33498.0,63532.0,"241,30",NaN,"15,10","21,40","44,00",NaN
